# 1.2 - External Data Collection - OMSCS Ratings

In [53]:
import pandas as pd
import json

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

%env GOOGLE_APPLICATION_CREDENTIALS=/Users/cesleemontgomery/masters/cs6460/CourseRec/courserec-adfe3-firebase-adminsdk-pm32q-6cd7a11195.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/cesleemontgomery/masters/cs6460/CourseRec/courserec-adfe3-firebase-adminsdk-pm32q-6cd7a11195.json


## 1. Load from JSON

In [42]:
file = '../../data/raw/reviews_omscentral/anonymized-reviews.json'
with open(file) as f:
    data = json.loads(f.read())
    omscentral_reviews_DF = pd.DataFrame(data['reviews'])

In [46]:
omscentral_reviews_DF.head(5)

,author,course,created,difficulty,extraCredit,firstCourse,frontLoad,groupProjects,id,moneySpent,previousClasses,proctortrack,program,projects,rating,semester,tests,text,updated,workload
0,google:115398662116989618541,CS-6300,1446786964000,2,NaN,NaN,NaN,NaN,-K2QDGcJfyyg5eiByfMR,NaN,NaN,NaN,NaN,NaN,5,2015-1,NaN,"This was one of the first courses I took, and ...",1.459894e+12,5.0
1,google:115398662116989618541,CS-6250,1447605174000,2,NaN,NaN,NaN,NaN,-K3AzV12uFLeqEV_MK03,NaN,NaN,NaN,NaN,NaN,4,2015-2,NaN,Interesting material. No exams. Just open-book...,1.476502e+12,3.0
2,google:115398662116989618541,CS-6505,1447605816000,5,NaN,NaN,NaN,NaN,-K3B0wpgLgZ1-X7ATVuC,NaN,NaN,NaN,NaN,NaN,1,2015-1,NaN,If you do not have a discrete mathematics back...,1.487453e+12,20.0
3,google:101531499641196700998,CSE-6220,1447867339000,4,NaN,NaN,NaN,NaN,-K3QbY3WczPdeGng4Ubz,NaN,NaN,NaN,NaN,NaN,5,2015-3,NaN,"Interesting lectures, great instructor. Challe...",1.466218e+12,18.0
4,7a83c83e-b098-4169-9aea-d20c0b151c37,CS-6035,1449296587000,2,NaN,NaN,NaN,NaN,-K4knimBGqR0wAMp0ZwA,NaN,NaN,NaN,NaN,NaN,4,2015-3,NaN,This has been an interesting course because I'...,1.471233e+12,3.0


In [52]:
omscentral_reviews_DF.to_csv('../../data/processed/course_reviews_omscentral.csv')

## 2. Transform - Aggregate Individual Ratings

### 2.1 - Aggregate Ratings - Mean

In [91]:
writedf.index

Index(['CS-6035', 'CS-6200', 'CS-6210', 'CS-6250', 'CS-6262', 'CS-6263',
       'CS-6290', 'CS-6291', 'CS-6300', 'CS-6310', 'CS-6340', 'CS-6400',
       'CS-6440', 'CS-6460', 'CS-6475', 'CS-6476', 'CS-6505', 'CS-6601',
       'CS-6750', 'CS-7637', 'CS-7641', 'CS-7642', 'CS-7646', 'CS-8803-001',
       'CS-8803-004', 'CS-8803-008', 'CS-8803-009', 'CS-8803-GA', 'CSE-6040',
       'CSE-6220', 'CSE-6242', 'CSE-6250', 'ISYE-6402', 'ISYE-6414',
       'ISYE-6501', 'MGT-6203', 'MGT-6754'],
      dtype='object', name='course')

In [99]:
writedf = omscentral_reviews_DF.groupby('course')[['difficulty','workload', 'rating']].apply(lambda col: col.astype('float').mean())
writedf['course_number'] = writedf.index.str.replace("-", " ") #for compatibility with course metadata source
writedf.set_index('course_number', inplace=True)
writedf.head()

,difficulty,workload,rating
course_number,,,
CS 6035,2.344444,8.770000,3.866667
CS 6200,3.537500,16.812500,4.500000
CS 6210,4.391304,17.347826,4.304348
CS 6250,2.853659,10.115854,3.670732
CS 6262,3.000000,11.250000,2.818182


### 2.2 - TODO - Create explanations of constraints

![Design](../../img/explanations-design.png)

In [ ]:
# One of the remaining features that I would like to have implemented is a detailed explanation table of 
# how each recommended course compared to others educational constraints (workload, quality, difficulty).
# Let's create a util function that accepts a selected course and computes a useful comparison to help students
# decide which course best meets their needs (may or may not look like above).

# Minimum this should have a numeric comparison statistic, string explanation, some visual cue (color, font effects, etc).

#Importantly, the nature of the comparison statistic is TBD.  Should it be point difference, percentiles, or something else?

## 3. Productionize - Write to Firebase

In [101]:
course_subs_DF = pd.read_csv('../../data/processed/course_video_subtitles_udacity.csv')
course_subs_DF.head()

,course_id,course_name,course_number,course_slug,course_link,subtitles
0,0,Artificial Intelligence for Robotics,CS 7638,Artificial+Intelligence+for+Robotics+,https://www.omscs.gatech.edu/cs-7638-artificia...,P P Proportional \n \n\n\n tau \n\n \n \...
1,1,Intro to High-Performance Computing,CSE 6220,High Performance Computing,https://www.omscs.gatech.edu/cse-6220-intro-hpc,Consider this network\nI have a bunch of quest...
2,2,NaN,NaN,"Computability, Complexity & Algorithms",NaN,Before moving on from this question of\nevalua...
3,3,Machine Learning,CS 7641,Machine Learning,https://www.omscs.gatech.edu/cs-7641-machine-l...,a b \n d \n\n d \n f g \n d \n\n a g \n\n\...
4,4,Database Systems Concepts and Design,CS 6400,Database+Systems+Concepts+&+Design+,https://www.omscs.gatech.edu/cs-6400-database-...,Sometimes tables are already indexed by the da...


In [107]:
writedf = pd.merge(course_subs_DF, writedf, on='course_number').drop(['subtitles', 'course_slug'], axis=1)
writedf.head()

,course_id,course_name,course_number,course_link,difficulty,workload,rating
0,1,Intro to High-Performance Computing,CSE 6220,https://www.omscs.gatech.edu/cse-6220-intro-hpc,4.500000,20.050000,4.800000
1,3,Machine Learning,CS 7641,https://www.omscs.gatech.edu/cs-7641-machine-l...,4.169811,20.849057,4.264151
2,4,Database Systems Concepts and Design,CS 6400,https://www.omscs.gatech.edu/cs-6400-database-...,2.651163,10.186047,3.325581
3,5,Knowledge-Based Artificial Intelligence: Cogni...,CS 7637,https://www.omscs.gatech.edu/cs-7637-knowledge...,3.044118,13.933824,3.750000
4,6,Educational Technology,CS 6460,https://www.omscs.gatech.edu/cs-6460-education...,2.777778,12.962963,4.703704


In [54]:
# Some help from: https://medium.com/@hmurari/cloud-firestore-batch-transactions-how-to-migrate-a-large-amounts-of-data-336e61efbe7c
# Use the application default credentials
# cred = credentials.ApplicationDefault()
# firebase_admin.initialize_app(cred, {
#   'projectId': 'courserec-adfe3',
# })

# db = firestore.client()

In [110]:
writedf.iloc[0]

course_id                                                      1
course_name                  Intro to High-Performance Computing
course_number                                           CSE 6220
course_link      https://www.omscs.gatech.edu/cse-6220-intro-hpc
difficulty                                                   4.5
workload                                                   20.05
rating                                                       4.8
Name: 0, dtype: object

In [114]:
courses_collection = db.collection(u'courses')

#for batching
total = len(writedf)
counter = 0

# Start a batch
batch = db.batch()
for idx, record in writedf.iterrows():
    # Commit the batch at every 500th record.
    if counter % 500 == 0:
        if counter > 0:
            print('Committing..')
            batch.commit()

        # Start a new batch for the next iteration.
        batch = db.batch()
    counter += 1
    print(str(counter) + str('/') + str(total) + ': ' + str(record.course_name))
    record_id = record.course_name
    record_ref = courses_collection.document(str(record_id))
    # Include current record in batch
    batch.set(record_ref, 
        {
           "id": record.course_id,
            "link": record.course_link,
            "name": record.course_name,
            "number": record.course_number,
            "ratingDifficulty": record.difficulty,
            "ratingQuality": record.rating,
            "ratingWorkload": record.workload
    })

# Include current record in batch
if counter % 500 != 0:
    print('Committing..')
    batch.commit()

1/25: Intro to High-Performance Computing
2/25: Machine Learning
3/25: Database Systems Concepts and Design
4/25: Knowledge-Based Artificial Intelligence: Cognitive Systems
5/25: Educational Technology
6/25: Introduction to Operating Systems
7/25: Computer Networks
8/25: Special Topics: Big Data for Health Informatics
9/25: Advanced Operating Systems
10/25: Software Analysis
11/25: Data and Visual Analytics
12/25: Computer Vision
13/25: High Performance Computer Architecture
14/25: Network Security
15/25: Artificial Intelligence
16/25: Intro to Health Informatics
17/25: Introduction to Information Security
18/25: Embedded Systems Optimization
19/25: Computational Photography
20/25: Machine Learning for Trading
21/25: Reinforcement Learning
22/25: Human-Computer Interaction
23/25: Software Development Process
24/25: Intro to Cyber Physical Systems Security
25/25: Software Architecture and Design
Committing..


In [115]:
writedf

,course_id,course_name,course_number,course_link,difficulty,workload,rating
0,1,Intro to High-Performance Computing,CSE 6220,https://www.omscs.gatech.edu/cse-6220-intro-hpc,4.500000,20.050000,4.800000
1,3,Machine Learning,CS 7641,https://www.omscs.gatech.edu/cs-7641-machine-l...,4.169811,20.849057,4.264151
2,4,Database Systems Concepts and Design,CS 6400,https://www.omscs.gatech.edu/cs-6400-database-...,2.651163,10.186047,3.325581
3,5,Knowledge-Based Artificial Intelligence: Cogni...,CS 7637,https://www.omscs.gatech.edu/cs-7637-knowledge...,3.044118,13.933824,3.750000
4,6,Educational Technology,CS 6460,https://www.omscs.gatech.edu/cs-6460-education...,2.777778,12.962963,4.703704
5,7,Introduction to Operating Systems,CS 6200,https://www.omscs.gatech.edu/cs-8803-introduct...,3.537500,16.812500,4.500000
6,8,Computer Networks,CS 6250,https://www.omscs.gatech.edu/cs-6250-computer-...,2.853659,10.115854,3.670732
7,9,Special Topics: Big Data for Health Informatics,CSE 6250,https://www.omscs.gatech.edu/cse-8803-special-...,4.722222,32.777778,3.666667
8,10,Advanced Operating Systems,CS 6210,https://www.omscs.gatech.edu/cs-6210-advanced-...,4.391304,17.347826,4.304348
9,12,Software Analysis,CS 6340,https://www.omscs.gatech.edu/cs-6340-software-...,3.590909,12.386364,3.204545
